In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import h5py
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import os

In [ ]:
file=h5py.File('../input/MNIST_Subset.h5','r')
images=file['X'][:]
labels=file['Y'][:]
images_train, images_test, labels_train, labels_test = train_test_split(images, labels, test_size=0.3, random_state=42)
print('test-image-shape',images_test.shape,',train-image-shape',images_train.shape,'test-label-shape',labels_test.shape,',train-label-shape',labels_train.shape)

In [ ]:
label_train=np.where(labels_train==7,0,1)
label_test=np.where(labels_test==7,0,1)
label_train=label_train.reshape(1,9975)
label_test=label_test.reshape(1,4276)
print(label_test.shape)
print(label_train.shape)
image_test=np.reshape(images_test,(784,4276))
image_train=np.reshape(images_train,(784,9975))
print(image_test.shape)
print(image_train.shape)

In [ ]:
# def initialize_params(layers_dims,m):
#     parameters={}
#     no_of_layers=len(layers_dims)
#     for i in range(1,no_of_layers):
#         parameters['W'+str(i)]=np.random.rand(layers_dims[i],layers_dims[i-1])/ np.sqrt(layers_dims[i-1])
#         parameters['b'+str(i)]=np.zeros((layers_dims[i],m))         
#     return parameters
# def softmax(a):
#     cache=a
#     softm=np.exp(a)
#     softm=softm/(np.sum(softm,keepdims=True))
#     return softm,cache
# def sigmoid(Z):
#     cache=Z
#     return 1.0/(1.0+np.exp(-Z)),cache
# def relu(Z):
#     cache=Z
#     return np.max(0,Z),cache
# def linear_forwrd(A_PREV,W,b):
# #     print('A_PREV',A_PREV.shape)
# #     print('W',W.shape)
# #     print('b',b.shape)
#     Z = np.add(np.matmul(W, A_PREV), b)
#     cache=(A_PREV,W,b)
# #     print('Z',Z.shape)
#     return Z,cache
# def linear_activation(A_PREV,W,b,activation):
#     if activation=="sigmoid":
#         Z,linear_cache=linear_forwrd(A_PREV,W,b)
# #         print('Z',Z.shape)
#         A,activation_cache=sigmoid(Z)
# #         print('A',A.shape)
#     elif activation=="relu":
#         Z,linear_cache=linear_forwrd(A_PREV,W,b)
#         A,activation_cache=relu(Z)
#     elif activation=="softmax":
#         Z,linear_cache=linear_forwrd(A_PREV,W,b)
#         A,activation_cache=softmax(Z)
#     cache=(linear_cache,activation_cache)
#     return A,cache       
# def forward_prop(images,parameters):
#     caches=[]
#     A=images
#     L=len(parameters)//2
#     for i in range(1,L+1):
#         if(i==L):
# #             print('called',i)
#             A_PREV=A   
# #             print('prev',(np.unique(A_PREV)))
#             A,cache=linear_activation(A_PREV,parameters['W'+str(i)],parameters['b'+str(i)],'sigmoid')
# #             print('new',np.unique(A))
#             caches.append(cache)
#         else:
# #             print(i)
#             A_PREV=A
# #             print('prev',np.unique(A_PREV))
#             A,cache=linear_activation(A_PREV,parameters['W'+str(i)],parameters['b'+str(i)],'sigmoid')
# #             print('new',np.unique(A))
#             caches.append(cache)
#     return A,caches


In [ ]:
def initialize_parameters_deep(layer_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the dimensions of each layer in our network
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
                    bl -- bias vector of shape (layer_dims[l], 1)
    """
    
    np.random.seed(1)
    parameters = {}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) / np.sqrt(layer_dims[l-1]) #*0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return parameters

def linear_forward(A_prev, W, b):
    """
    Implement the linear part of a layer's forward propagation.

    Arguments:
    A_prev -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)

    Returns:
    Z -- the input of the activation function, also called pre-activation parameter 
    cache -- a python dictionary containing "A", "W" and "b" ; stored for computing the backward pass efficiently
    """
    
    Z = np.add(np.matmul(W, A_prev), b)
    
    assert(Z.shape == (W.shape[0], A_prev.shape[1]))
    cache = (A_prev, W, b)
    
    return Z, cache

def sigmoid(Z):
    """
    Implements the sigmoid activation in numpy
    
    Arguments:
    Z -- numpy array of any shape
    
    Returns:
    A -- output of sigmoid(z), same shape as Z
    cache -- returns Z as well, useful during backpropagation
    """
    
    A = 1/(1+np.exp(-Z))
    cache = Z
    
    return A, cache

def softmax(a):
    cache=a
    softm=np.exp(a-np.max(a))
    softm=softm/(np.sum(softm,keepdims=True))
    return softm,cache

def relu(Z):
    """
    Implement the RELU function.

    Arguments:
    Z -- Output of the linear layer, of any shape

    Returns:
    A -- Post-activation parameter, of the same shape as Z
    cache -- a python dictionary containing "A" ; stored for computing the backward pass efficiently
    """
    
    A = np.maximum(0,Z)
    
    assert(A.shape == Z.shape)
    
    cache = Z 
    return A, cache

## Linear-Activation Forward
def linear_activation_forward(A_prev, W, b, activation):
    """
    Implement the forward propagation for the LINEAR->ACTIVATION layer

    Arguments:
    A_prev -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    A -- the output of the activation function, also called the post-activation value 
    cache -- a python dictionary containing "linear_cache" and "activation_cache";
             stored for computing the backward pass efficiently
    """
    
    if activation == "sigmoid":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        ### START CODE HERE ### (≈ 2 lines of code)
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
        ### END CODE HERE ###
    
    elif activation == "softmax":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        ### START CODE HERE ### (≈ 2 lines of code)
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = softmax(Z)
        ### END CODE HERE ###
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache

def L_model_forward(X, parameters):
    """
    Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation
    
    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize_parameters_deep()
    
    Returns:
    AL -- last post-activation value
    caches -- list of caches containing:
                every cache of linear_activation_forward(A_prev, W, b, 'relu') (there are L-1 of them, indexed from 0 to L-2)
                the cache of linear_activation_forward(A_prev, W, b, 'sigmoid') (there is one, indexed L-1)
    """

    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network
    
    # Implement [LINEAR -> RELU]*(L-1). Add "cache" to the "caches" list.
    for l in range(1, L):
        A_prev = A 
        A, cache = linear_activation_forward(
            A_prev, parameters['W' + str(l)], parameters['b' + str(l)], 'sigmoid'
        )
        caches.append(cache)
    
    # Implement LINEAR -> SIGMOID. Add "cache" to the "caches" list.
    AL, cache = linear_activation_forward(
            A, parameters['W' + str(L)], parameters['b' + str(L)], 'sigmoid'
        )
    caches.append(cache)

    assert(AL.shape == (1,X.shape[1]))
            
    return AL, caches


def compute_cost(AL, Y):
    """
    Implement the cost function defined by equation (7).

    Arguments:
    AL -- probability vector corresponding to your label predictions, shape (1, number of examples)
    Y -- true "label" vector (for example: containing 0 if non-cat, 1 if cat), shape (1, number of examples)

    Returns:
    cost -- cross-entropy cost
    """
    
    m = Y.shape[1]
   # print(AL.shape,Y.shape)
    # Compute loss from aL and y.
    cost = (1./m) * (-np.dot(Y,np.log(AL).T) - np.dot(1-Y, np.log(1-AL).T))
    
    cost = np.squeeze(cost)  
#     print(cost.shape)
    # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    assert(cost.shape == ())
    
    return cost

In [ ]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    dW = (1.0/m) * np.matmul(dZ, A_prev.T)
    db = (1.0/m) * np.sum(dZ, axis=-1, keepdims=True)
    dA_prev = np.matmul(np.transpose(W), dZ) 
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)    
    return dA_prev, dW, db
def softmax_backward(softmax,cache):
    # Reshape the 1-d softmax to 2-d so that np.dot will do the matrix multiplication
    s = softmax.reshape(-1,1)
    return np.diagflat(s) - np.dot(s, s.T),cache
def sigmoid_backward(dA, cache):
    Z = cache
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    return dZ
def relu_backward(dA, cache):
    Z = cache
    dZ = np.array(dA, copy=True) # just converting dz to a correct object.
    dZ[Z <= 0] = 0
    assert (dZ.shape == Z.shape) 
    return dZ
def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache   
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)        
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)  
    elif activation=="softmax":
        dZ = softmax_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)  
    return dA_prev, dW, db
def L_model_backward(AL, Y, caches):
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
#     print(m,AL.shape,L,Y.shape)
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL    
    # Initializing the backpropagation
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))    
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "AL, Y, caches". Outputs: "grads["dAL"], grads["dWL"], grads["dbL"]
    current_cache = caches[L-1]
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation = "sigmoid")    
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 2)], current_cache, activation = "sigmoid")
        grads["dA" + str(l + 1)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
    return grads
def update_parameters(parameters, grads, learning_rate):
    L = len(parameters) // 2 # number of layers in the neural network
    # Update rule for each parameter. Use a for loop.
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]        
    return parameters
def L_layer_model(X, Y, layers_dims, parameters=None, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):#lr was 0.009
    np.random.seed(1)
    costs = []                         # keep track of cost
    if not parameters:
        parameters = initialize_parameters_deep(layers_dims)
    for i in range(0, num_iterations):
        AL, caches = L_model_forward(X, parameters)
        # Compute cost.
       # print('AL',AL.shape)
        cost = compute_cost(AL, Y)
        # Backward propagation.
        grads = L_model_backward(AL, Y, caches)       
        # Update parameters.
        parameters = update_parameters(parameters, grads, learning_rate)                
        # Print the cost every 100 training example
        if print_cost and i % 10 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)            
    # plot the cost
    plt.plot(costs)
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()    
    return parameters, costs
layers_dims=[784,100,1]
parameters, costs = L_layer_model(image_train,label_train, layers_dims, num_iterations = 3000, print_cost = True)

In [ ]:
def predict(X, y, parameters):
    """
    This function is used to predict the results of a  L-layer neural network.
    
    Arguments:
    X -- data set of examples you would like to label
    parameters -- parameters of the trained model
    
    Returns:
    p -- predictions for the given dataset X
    """
    
    m = X.shape[1]
    n = len(parameters) // 2 # number of layers in the neural network
    p = np.zeros((1,m))
    
    # Forward propagation
    probas, caches = L_model_forward(X, parameters)

    
    # convert probas to 0/1 predictions
    for i in range(0, probas.shape[1]):
        if probas[0,i] > 0.5:
            p[0,i] = 1
        else:
            p[0,i] = 0
    
    #print results
    #print ("predictions: " + str(p))
    #print ("true labels: " + str(y))
    print("Accuracy: "  + str(np.sum((p == y)/m)))
        
    return p
pred_train = predict(image_train,label_train, parameters)
pred_test = predict(image_test,label_test, parameters)

In [ ]:
layers_dims=[784,100,50,50,1]
parameters, costs = L_layer_model(image_train,label_train, layers_dims, num_iterations = 3000, print_cost = True)

In [ ]:
pred_train = predict(image_train,label_train, parameters)
pred_test = predict(image_test,label_test, parameters)

In [ ]:
# def print_mislabeled_images(classes, X, y, p):
#     """
#     Plots images where predictions and truth were different.
#     X -- dataset
#     y -- true labels
#     p -- predictions
#     """
#     a = p + y
#     mislabeled_indices = np.asarray(np.where(a == 1))
#     plt.rcParams['figure.figsize'] = (40.0, 40.0) # set default size of plots
#     num_images = len(mislabeled_indices[0])
#     for i in range(num_images):
#         index = mislabeled_indices[1][i]
#         plt.subplot(2, num_images, i + 1)
#         plt.imshow(X[:,index].reshape(28,28), interpolation='nearest')
#         plt.axis('off')
#         plt.title("Prediction: " + classes[int(p[0,index])].decode("utf-8") + " \n Class: " + classes[y[0,index]].decode("utf-8"))
# classes = np.array([b'7',b'9'])
# print_mislabeled_images(classes, image_train, label_train, pred_train)

In [ ]:
# # for relu
# def forward_prop(images,parameters):
#     A=images
#     L=len(parameters)//2
#     for i in range(1,L+1):
#         if(i==L):
#             print('called',i)#,parameters['W'+str(i)].shape,parameters['b'+str(i)].shape)
#             A_PREV=A   
#             print('prev',(np.unique(A_PREV)))
#             A=linear_activation(A_PREV,parameters['W'+str(i)],parameters['b'+str(i)],'softmax')
#             print('new',A)
#         else:
#             print(i)#parameters['W'+str(i)].shape,parameters['b'+str(i)].shape)
# #             print(A)
#             A_PREV=A
#             print('prev',(np.unique(A_PREV)))
#             A=linear_activation(A_PREV,parameters['W'+str(i)],parameters['b'+str(i)],'relu')
#             print('new',np.unique(A))
#     return A
# A=forward_prop(image_train,parameters)
# #print(A)
    

In [ ]:
import torch

In [ ]:
import torchvision.models as models
alexnet = models.alexnet(pretrained=True)